In [1]:
# install the open cv library
!pip install opencv-python

In [48]:
# importing the packages
import pandas as pd
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [3]:
# there are approx 5 features present overall across 75000 images
features_dataframe = pd.read_csv("final_dataset_containing_5_extracted_features.csv")

In [4]:
features_dataframe.shape

(72986, 12)

In [5]:
# viewing the features_dataframe
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
0,00000.png,125,101.0,73.0,NaN,NaN,89,130.0,87.0,NaN,NaN,No
1,000004.png,127,75.0,NaN,NaN,NaN,90,87.0,NaN,NaN,NaN,Yes
2,00001.png,126,75.0,76.0,NaN,NaN,91,90.0,139.0,NaN,NaN,No
3,000014.png,130,77.0,NaN,NaN,NaN,89,91.0,NaN,NaN,NaN,Yes
4,000016.png,127,173.0,74.0,NaN,NaN,90,61.0,91.0,NaN,NaN,Yes


In [6]:
# performing shuffling on features_dataframe
features_dataframe = features_dataframe.sample(frac=1)
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
69682,50511.png,82,124.0,120.0,NaN,NaN,137,90.0,135.0,NaN,NaN,No
64504,45062.png,78,126.0,108.0,121.0,81.0,87,90.0,136.0,72.0,137.0,No
57427,37628.png,124,77.0,102.0,NaN,NaN,87,89.0,139.0,NaN,NaN,No
13309,04014.png,125,96.0,76.0,102.0,115.0,89,113.0,89.0,132.0,27.0,No
23571,07095.png,80,126.0,NaN,NaN,NaN,90,89.0,NaN,NaN,NaN,No


In [7]:
features_dataframe = features_dataframe.fillna(-1)

In [8]:
# normalizing the data
is_gan = features_dataframe["is_gan"] 
features_dataframe.drop(columns=["file_name", "is_gan"], inplace=True)

In [9]:
standard_scalar = StandardScaler()
features_dataframe = standard_scalar.fit_transform(features_dataframe)
features_dataframe = pd.DataFrame(features_dataframe)
features_dataframe

,0,1,2,3,4,5,6,7,8,9
0,-1.033702,0.985635,1.479135,-0.432465,-0.189726,2.674522,0.156418,1.328066,-0.420531,-0.183362
1,-1.196018,1.038130,1.240842,2.818546,4.213639,-0.370958,0.156418,1.344442,1.216339,6.372390
2,0.670618,-0.247994,1.121696,-0.432465,-0.189726,-0.370958,0.127754,1.393569,-0.420531,-0.183362
3,0.711197,0.250707,0.605394,2.312241,6.039424,-0.249139,0.815698,0.574783,2.561710,1.146791
4,-1.114860,1.038130,-0.923653,-0.432465,-0.189726,-0.188229,0.127754,-0.899031,-0.420531,-0.183362
...,...,...,...,...,...,...,...,...,...,...
72981,-1.236597,1.064378,1.022407,-0.432465,-0.189726,-0.188229,0.070425,1.377193,-0.420531,-0.183362
72982,0.792355,-0.300489,-0.923653,-0.432465,-0.189726,-0.249139,0.127754,-0.899031,-0.420531,-0.183362
72983,0.792355,-1.166654,0.585536,-0.432465,-0.189726,-0.249139,-0.932827,0.607535,-0.420531,-0.183362
72984,-1.358334,0.959388,0.585536,-0.432465,-0.189726,-0.188229,0.156418,0.280020,-0.420531,-0.183362


In [10]:
features_dataframe["is_gan"] = is_gan
features_dataframe["is_gan"] = features_dataframe["is_gan"].map(dict(Yes=1, No=0))
features_dataframe

,0,1,2,3,4,5,6,7,8,9,is_gan
0,-1.033702,0.985635,1.479135,-0.432465,-0.189726,2.674522,0.156418,1.328066,-0.420531,-0.183362,0
1,-1.196018,1.038130,1.240842,2.818546,4.213639,-0.370958,0.156418,1.344442,1.216339,6.372390,1
2,0.670618,-0.247994,1.121696,-0.432465,-0.189726,-0.370958,0.127754,1.393569,-0.420531,-0.183362,0
3,0.711197,0.250707,0.605394,2.312241,6.039424,-0.249139,0.815698,0.574783,2.561710,1.146791,1
4,-1.114860,1.038130,-0.923653,-0.432465,-0.189726,-0.188229,0.127754,-0.899031,-0.420531,-0.183362,1
...,...,...,...,...,...,...,...,...,...,...,...
72981,-1.236597,1.064378,1.022407,-0.432465,-0.189726,-0.188229,0.070425,1.377193,-0.420531,-0.183362,0
72982,0.792355,-0.300489,-0.923653,-0.432465,-0.189726,-0.249139,0.127754,-0.899031,-0.420531,-0.183362,0
72983,0.792355,-1.166654,0.585536,-0.432465,-0.189726,-0.249139,-0.932827,0.607535,-0.420531,-0.183362,0
72984,-1.358334,0.959388,0.585536,-0.432465,-0.189726,-0.188229,0.156418,0.280020,-0.420531,-0.183362,0


In [11]:
features_dataframe.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
is_gan    0
dtype: int64

In [12]:
x = features_dataframe
y = features_dataframe["is_gan"]
x.drop(columns=["is_gan"], inplace=True)

In [67]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [68]:
f"Shape of training data = {x_train.shape[0]} and testing data = {x_test.shape[0]}"

'Shape of training data = 51090 and testing data = 21896'

## Performing experiments using the following hyperparameters to find the best model

In [57]:
# Performing Kfold cross validation
logit_pipeline = Pipeline([("classifier", LogisticRegression())])
param_grid = {'classifier__penalty' : ['l1', 'l2', 'elasticnet'],
             'classifier__C' : [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1],
             'classifier__solver' : ["saga"],
             'classifier__max_iter': [50, 100, 150, 200],
             'classifier__n_jobs': [-1]}

logit_pipeline

Pipeline(steps=[('classifier', LogisticRegression())])

In [58]:
gridsearch_cv = GridSearchCV(logit_pipeline, param_grid=param_grid, cv=10, verbose=2)

In [59]:
gridsearch_cv.fit(x_train, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   2.4s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.9s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.9s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.8s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.8s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.9s
[CV] END classifier__C=0.05, classifier__max_iter=50, classifier_

[CV] END classifier__C=0.05, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.2s
[CV] END classifier__C=0.05, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.2s
[CV] END classifier__C=0.05, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.3s
[CV] END classifier__C=0.05, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.5s
[CV] END classifier__C=0.05, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   2.4s
[CV] END classifier__C=0.05, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l1, classifier__solver=saga; total time=   0.9s
[CV] END classifier__C=0.05, classifier__max_iter=150, classifier__n_jobs=-1, classifier__pe

[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.4s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.2s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.2s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.1s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.5s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.3s
[CV] END classifier__C=0.05, classifier__max_iter=200, classifier__n_jobs=-1

[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.1, classifier__max_iter=100, classifier__n_jobs=-1, classifier__penalty=e

[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=0.1, classifier__max_iter=200, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=sa

...

[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   0.1s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.1s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.1s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.0s
[CV] END classifier__C=0.95, classifier__max_iter=150, classifier__n_jobs=-1

[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   0.1s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; total time=   2.3s
[CV] END classifier__C=1, classifier__max_iter=50, classifier__n_jobs=-1, classifier__penalty=elasticnet, classifier__solver=saga; t

[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.2s
[CV] END classifier__C=1, classifier__max_iter=150, classifier__n_jobs=-1, classifier__penalty=l2, classifier__solver=saga; total time=   0.3s

C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
800 fits failed out of a total of 2400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
800 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 436, in _process_worker
    r = call_item()
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 288, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\si

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3,
                                           0.35, 0.4, 0.45, 0.5, 0.55, 0.6,
                                           0.65, 0.7, 0.75, 0.8, 0.85, 0.9,
                                           0.95, 1],
                         'classifier__max_iter': [50, 100, 150, 200],
                         'classifier__n_jobs': [-1],
                         'classifier__penalty': ['l1', 'l2', 'elasticnet'],
                         'classifier__solver': ['saga']},
             verbose=2)

In [60]:
gridsearch_cv.best_params_

{'classifier__C': 0.05,
 'classifier__max_iter': 50,
 'classifier__n_jobs': -1,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

## Training the model on the above best parameters

In [79]:
logisticRegr = LogisticRegression(C=0.05, max_iter=50, n_jobs=-1, penalty="l2", solver="saga")
logisticRegr.fit(x_train, y_train)

LogisticRegression(C=0.05, max_iter=50, n_jobs=-1, solver='saga')

In [80]:
logit_predictions = logisticRegr.predict(x_test)

In [81]:
cm = metrics.confusion_matrix(y_test, logit_predictions)
print(f"The confusion matrix for Logistic Regression is given below: \n {cm}")

The confusion matrix for Logistic Regression is given below: 
 [[14908     0]
 [ 6988     0]]


In [82]:
logit_precision = precision_score(y_test, logit_predictions, average='weighted', labels=np.unique(logit_predictions))
logit_precision

0.6808549506759225

In [83]:
logit_recall = recall_score(y_test, logit_predictions, average='weighted', labels=np.unique(logit_predictions))
logit_recall

1.0

In [84]:
logit_f1 = f1_score(y_test, logit_predictions, average='weighted', labels=np.unique(logit_predictions))
logit_f1

0.8101293337680686

In [85]:
# storing the svm model
filename = 'logistic_regression_final.pkl'
pickle.dump(logisticRegr, open(filename, 'wb'))